In [7]:
#Uncomment these lines to install libraries for this script

#!pip install -e git+https://github.com/SohierDane/BigQuery_Helper#egg=bq_helper
#!pip install --upgrade google-cloud-bigquery

In [36]:
import os
import bq_helper
import datetime as datetime
from bq_helper import BigQueryHelper
import pandas as pd
%run country-boundingbox.ipynb

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "Data-mining-afa11af25388.json"
# https://www.kaggle.com/sohier/introduction-to-the-bq-helper-package
noaa = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                   dataset_name="ghcn_d")

In [37]:
#Function returns all weather station data within the country between the specified years. Inefficient, but  complete.
#All country IDs can be found in the country-boundingbox file

def weatherDataByCountry(countryID, startYear, endYear):
    coordinates = country_bounding_boxes[countryID][1]
    lonMin = str(coordinates[0])
    latMin = str(coordinates[1])
    lonMax = str(coordinates[2])
    latMax = str(coordinates[3])
    weatherData = pd.DataFrame()
    for i in range(startYear,endYear):
        query = """SELECT
          stations.id,
          weather.date,
          weather.element,
          weather.value
        FROM
          `bigquery-public-data.ghcn_d.ghcnd_"""+str(i)+"""` AS weather INNER JOIN `bigquery-public-data.ghcn_d.ghcnd_stations` AS stations ON 
          weather.id = stations.id
        WHERE
          weather.element = 'TAVG'
          AND stations.latitude >"""+latMin+"""
          AND stations.latitude <"""+latMax+"""
          AND stations.longitude >"""+lonMin+"""
          AND stations.longitude <"""+lonMax+"""
        ORDER BY
            weather.date ASC;
                """
        response2 = noaa.query_to_pandas_safe(query, max_gb_scanned=10)
        weatherData = weatherData.append(response2)
    weatherData.dropna()
    return weatherData

In [38]:
dfWeather= weatherDataByCountry('HN', 2013, 2015)
print(dfWeather.size)
print(dfWeather.head(10))

13964
            id        date element  value
0  HOM00078705  2013-01-01    TAVG  267.0
1  ES003800510  2013-01-01    TAVG  263.0
2  HO000078720  2013-01-01    TAVG  194.0
3  HOM00078708  2013-01-01    TAVG  249.0
4  ES003800510  2013-01-02    TAVG  262.0
5  HOM00078705  2013-01-02    TAVG  253.0
6  HOM00078708  2013-01-02    TAVG  261.0
7  HO000078720  2013-01-02    TAVG  209.0
8  HO000078711  2013-01-02    TAVG  259.0
9  HOM00078708  2013-01-03    TAVG  252.0


In [41]:
#Function finds the closest weather station to a specified coordinates and returns weather data between the specified years.
#Function takes in latitude and longitude coordinates (ensure to add negative signs to coordinates if necessary)

def weatherDataByCoordinates(lat, long, startYear, endYear):
    stationID = closestStationToLoc(lat, long)
    df = weatherDataForStationID(stationID, startYear, endYear)
    df['value'] = df['value'].div(10,axis=0) #fix value
    df["date"] = pd.to_datetime(df["date"]) #set Date as index
    df.set_index('date', inplace=True)
    return df

def prcpDataByCoordinates(lat, long, startYear, endYear):
    stationID = closestStationToLoc(lat, long)
    df = weatherDataForStationID(stationID, startYear, endYear)
    df['value'] = df['value'].div(10,axis=0) #fix value
    df["date"] = pd.to_datetime(df["date"]) #set Date as index
    df.set_index('date', inplace=True)
    return df


#Helper functions
def closestStationToLoc(lat, long):
    query = """SELECT weather.id, POWER(POWER(weather.latitude - """+str(lat)+""",2) + POWER(weather.longitude - """+str(long)+""",2), 1/2) AS distance, weather.latitude, weather.longitude
    FROM
      `bigquery-public-data.ghcn_d.ghcnd_stations` AS weather
    ORDER BY
        distance ASC
    LIMIT 10;
            """
    response2 = noaa.query_to_pandas_safe(query, max_gb_scanned=10)
    return response2['id'].iloc[0]

def weatherDataForStationID(ID, startYear, endYear):
    weatherData = pd.DataFrame()
    for i in range(startYear,endYear):
        query = """SELECT
          weather.date,
          weather.value
        FROM
          `bigquery-public-data.ghcn_d.ghcnd_"""+str(i)+"""` AS weather
        WHERE
            weather.element = 'TAVG'
            AND weather.id = '"""+ID+"""'
        ORDER BY
            weather.date ASC;
                """
        response2 = noaa.query_to_pandas_safe(query, max_gb_scanned=10)
        weatherData = weatherData.append(response2)
    weatherData.dropna()
    return weatherData

def prcpDataForStationID(ID, startYear, endYear):
    weatherData = pd.DataFrame()
    for i in range(startYear,endYear):
        query = """SELECT
          weather.date,
          weather.value
        FROM
          `bigquery-public-data.ghcn_d.ghcnd_"""+str(i)+"""` AS weather
        WHERE
            weather.element = 'PRCP'
            AND weather.id = '"""+ID+"""'
        ORDER BY
            weather.date ASC;
                """
        response2 = noaa.query_to_pandas_safe(query, max_gb_scanned=10)
        weatherData = weatherData.append(response2)
    weatherData.dropna()
    return weatherData

In [43]:
dfWeather = weatherDataByCoordinates(15.2000, -86.2419, 1990, 2017)
pdfWeather = prcpDataByCoordinates(15.2000, -86.2419, 1990, 2017)

##east and South are negitive
print(dfWeather.size)
print(dfWeather.head(10))
print(dfWeather.dtypes)

print(pdfWeather.size)
print(pdfWeather.head(10))
print(pdfWeather.dtypes)

9425
            value
date             
1990-01-01   24.4
1990-01-02   24.4
1990-01-03   22.7
1990-01-04   23.9
1990-01-05   25.4
1990-01-06   24.8
1990-01-07   23.9
1990-01-08   22.9
1990-01-09   24.6
1990-01-10   23.6
value    float64
dtype: object
9425
            value
date             
1990-01-01   24.4
1990-01-02   24.4
1990-01-03   22.7
1990-01-04   23.9
1990-01-05   25.4
1990-01-06   24.8
1990-01-07   23.9
1990-01-08   22.9
1990-01-09   24.6
1990-01-10   23.6
value    float64
dtype: object


In [46]:
#Turns a dataframe to the moving average with (i days)
# df - Pandas Dataframe
# i - Value for how many days to take the moving average with respect to
def movingAvg(df,i):
    if(i < 1):
        raise Exception('Invalid Int: Cannot find moving average of {}.'.format(sample))
    df['value'] = df['value'].rolling(i,center=True,min_periods=1).mean()
    return df

In [47]:
dfWeather_ma = movingAvg(dfWeather,3)
print(dfWeather_ma.head(10))

                value
date                 
1990-01-01  24.400000
1990-01-02  23.833333
1990-01-03  23.666667
1990-01-04  24.000000
1990-01-05  24.700000
1990-01-06  24.700000
1990-01-07  23.866667
1990-01-08  23.800000
1990-01-09  23.700000
1990-01-10  23.900000


In [62]:
#function to downsample to year month
def downSample(df,sample):
    if(sample == '1Y'):
        df= df.resample(sample).mean()
        df.index = df.index.to_period('y')
    elif(sample == '1M'):
        df= df.resample(sample).mean()
        df.index = df.index.to_period('m')
    else:
        raise Exception('Invalid Sample String: The value was: {}.'.format(sample))
    return df

def downSamplemin(df,sample):
    if(sample == '1Y'):
        df= df.resample(sample).min()
        df.index = df.index.to_period('y')
    elif(sample == '1M'):
        df= df.resample(sample).min()
        df.index = df.index.to_period('m')
    else:
        raise Exception('Invalid Sample String: The value was: {}.'.format(sample))
    return df

def downSamplemax(df,sample):
    if(sample == '1Y'):
        df= df.resample(sample).max()
        df.index = df.index.to_period('y')
    elif(sample == '1M'):
        df= df.resample(sample).max()
        df.index = df.index.to_period('m')
    else:
        raise Exception('Invalid Sample String: The value was: {}.'.format(sample))
    return df

In [70]:
#DownSample by Month
dfWeather_mo = downSample(dfWeather_ma,'1M')
#print(dfWeather_mo.head(10))

#Downsample by Year
dfWeather_yr = downSample(dfWeather_ma,'1Y')
dfWeather_yr = dfWeather_yr.rename(index=str,columns={"value":"tmpAvg"})
#print(dfWeather_yr)

downSamplemin_yr = downSamplemin(pdfWeather,'1Y')
downSamplemin_yr = downSamplemin_yr.rename(index=str,columns={"value":"prcpMin"})
#print(downSamplemin_yr)

downSamplemax_yr =downSamplemax(pdfWeather,'1Y')
downSamplemax_yr = downSamplemax_yr.rename(index=str,columns={'value':'prcpMax'})
#print(downSamplemax_yr)

merge=pd.merge(downSamplemin_yr,downSamplemax_yr, how='inner', left_index=True, right_index=True)
merge=pd.merge(merge,dfWeather_yr, how='inner', left_index=True, right_index=True)
print(merge)

      prcpMin  prcpMax     tmpAvg
date                             
1990     18.3     31.3  25.910214
1991     20.2     31.9  26.060762
1992     20.4     31.9  26.995308
1993     18.8     30.7  26.159890
1994     19.5     32.9  27.932738
1995     19.8     33.7  27.622011
1996     18.1     31.6  26.692197
1997     19.2     31.9  27.739268
1998     19.0     32.9  27.589169
1999     19.2     31.9  26.560231
2000     19.5     30.6  25.769367
2001     19.6     30.5  25.525023
2002     20.0     31.3  25.933991
2003     18.7     29.5  25.553333
2004     19.1     30.4  25.759270
2005     20.0     31.0  26.741738
2006     18.7     30.8  25.944539
2007     20.3     30.9  25.447688
2008     19.5     31.1  25.220591
2009     19.1     31.7  25.743970
2010     17.3     31.1  25.303693
2011     20.4     29.7  25.893973
2012     19.3     29.7  26.212750
2013     18.8     30.3  25.844197
2014     19.7     28.9  25.512785
2015     20.2     31.6  26.626301
2016     18.8     31.6  27.406639


In [60]:
m =0
i = 0
#for unique yearly key in dataframe
#Find max and set 
#
#
#
#
#
#
#
#
#
#
#
#

In [28]:
#OLS stuff
honduras = dfWeather_yr.to_csv("HondurasWeather1990-2016.csv",sep=',')